### Import Necessary Module and Set Path 

In [4]:
import sqlite3 as lite
import os
import sys
import collections
sys.path.insert(0, '../../scripts/')
jupyter_directory = os.getcwd()

In [5]:
import analysis as an
import random_lifetime_extension as rle
import cyclus_input_gen.from_pris as fp
import cyclus_input_gen.demand_deploy as dd

Usage: from_pris.main([csv],[init_date], [duration], [country_list], [output_file_name])


In [6]:
# also do random lifetime extension of French LWRs to see spread out deployment
# uniform sample (0,25)
rle.generate_input('input.xml', 'rand_ext.xml', 720, 'France', low=0, high=25)

        <name>France_government</name>



In [ ]:
! rm rand.sqlite
! cyclus rand_ext.xml -o rand.sqlite

In [ ]:
# get sqlite cursor and simulation time data
cur = an.cursor('rand.sqlite')
init_year, init_month, duration, timestep = an.simulation_timesteps(cur)

In [ ]:
# just france LWR power
power_capacity = an.power_capacity(cur)

power_demand_eq = str(max(power_capacity['France_government']))
print(power_demand_eq)
# with this power capacity, calculate deployment scheme for astrids:
astrid_lifetime = 720
astrid_deployment, powz = dd.get_new_deployment(power_capacity, ['France_government'],
                                                power_demand_eq, 0.6, astrid_lifetime, 840, True)
dd.write_deployinst(astrid_deployment, 'astrid', 'rand_astrid_dep.xml', astrid_lifetime)

In [ ]:
# add institution wrapper to generated deployinst block
head = """
      <institution>
        <name>France_astrid</name>
        <config>
          <DeployInst>
"""
tail = """

          </DeployInst>
        </config>
      </institution>
"""
with open('rand_astrid_dep.xml') as f:
    read_string = f.read()
    read_string = read_string.replace('<root>', '')
    read_string = read_string.replace('</root>', '')
    tot_string = head + read_string + tail
with open('rand_astrid_dep.xml', 'w') as f:
    f.write(tot_string)

In [ ]:
# insert astrid block
with open('rand_ext.xml') as f:
    input_string = f.read()

insert_start = """          </DeployInst>
        </config>
      </institution>"""


insert_end = """</region>
  
  <region>
      <name>Finland</name>
      <config><NullRegion/></config>
"""

new_input = replace(input_string, insert_start, insert_end, 'rand_astrid_dep.xml', insert=True)
print(len(new_input))
with open('rand_input_astrid.xml', 'w') as f:
    f.write(new_input)

In [ ]:
# run the new input with astrid reactors
! rm rand_ext_output.sqlite
! cyclus rand_input_astrid.xml -o rand_ext_output.sqlite

In [ ]:
# get sqlite cursor and simulation time data
cur = an.cursor('rand_ext_output.sqlite')
init_year, init_month, duration, timestep = an.simulation_timesteps(cur)

In [ ]:
power_cap = an.power_capacity(cur)
dictionary = {'lwr': power_cap['France_government'],
              'astrid': power_cap['France_astrid']}
init_year, init_month, duration, timestep = an.simulation_timesteps(cur)
an.stacked_bar_chart(dictionary, timestep,
                  'Years', 'Net_Capacity [GWe]',
                  'Net Capacity vs Time',
                  'rand_ext_france_power_plot', init_year)

In [ ]:
# astrid deployment
entry = cur.execute('SELECT entertime FROM agententry WHERE prototype == "astrid"').fetchall()
num_dict = {}
cum_dict = {}
num_dict['num dep'] = []
for t in timestep:
    astrid_num = 0
    for agent in entry:
        if agent['entertime'] == t:
            astrid_num += 1
    num_dict['num dep'].append(astrid_num)
dep = an.deployments(cur)
cum_dict['cumulative'] = dep['France_astrid']
plt.plot(init_year + timestep/12, num_dict['num dep'])
plt.show()
plt.close()
plt.plot(cum_dict['cumulative'])
print(max(cum_dict['cumulative']))

print('Average number of ASTRIDS to deploy per year')
print(sum(num_dict['num dep']) / (2160-2040) )
an.double_axis_bar_line_plot(num_dict, cum_dict, timestep,
                             'Years', 'Number of SFRs Deployed', ' Number of SFRs Operational',
                             'Build number and cumulative ASTRID Number vs Time', 'rand_sfr_deploy',
                             init_year)